# Imports 

In [ ]:
import os
import random 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, Flatten, Dropout, Input, GlobalMaxPooling1D, concatenate

sacando codigos de folder

In [ ]:
path = "C:/Users/Alex/Documents/Cisco Config/Algorithms/Reto/reto/Dataset/version_2"

# Lista de carpetas en el dataset original
folder_group = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

# DataFrame 
Definimos nuestro DataFrame, Path, y empezamos con la lectura de los archivos de Java con UTF-8. El DataFrame contendra los codigos a procesar.

In [ ]:
df = pd.DataFrame(columns=['Nombre1','Code1','Nombre2', 'Code2'])

In [ ]:
import os
import pandas as pd

path = "C:/Users/Alex/Documents/Cisco Config/Algorithms/Reto/reto/Dataset/version_2"

# Lista de carpetas en el dataset original
folder_group = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

df = pd.DataFrame(columns=['Nombre1','Code1','Nombre2', 'Code2'])

for folder in folder_group:
    dataset1 = []
    dataset2 = []

    folder_path = os.path.join(path, folder)
    problem_list = os.listdir(folder_path)

    # Eliminar extensión ".java" del nombre del archivo
    problem_list = [os.path.splitext(file)[0] for file in problem_list]

    # Leer el primer archivo .java con UTF-8
    with open(os.path.join(folder_path, problem_list[0] + '.java'), 'r', encoding='utf-8') as file1:
        dataset1.append(file1.read())
    d1 = ' '.join(dataset1)

    # Leer el segundo archivo .java con UTF-8
    with open(os.path.join(folder_path, problem_list[1] + '.java'), 'r', encoding='utf-8') as file2:
        dataset2.append(file2.read())
    d2 = ' '.join(dataset2)

    # Añadir datos al DataFrame
    df.loc[len(df.index)] = [problem_list[0], d1, problem_list[1], d2]


# Processamiento de Texto
Cargamos la informacion de nuestro dataset y verificamos que se cargen correctamente.

In [ ]:
def load_data_from_directory(directory):
    texts = []
    labels = []
    # print(directory)
    for label in os.listdir(directory):
        # print(label)
        # class_dir = os.path.join(directory, label)
        class_dir = os.path.join(directory)
        # print(class_dir)
        if label.endswith(".txt"):
            file_path = os.path.join(class_dir, label)
            # print(file_path)
            with open(file_path, 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(label)
                # print(texts)
    return texts, labels

# Ruta a la carpeta que contiene las subcarpetas 'java', 'python', y 'c'
data_directory = 'C:/Users/Alex/Documents/cisco Config/Algorithms/Reto/reto/reto'  # Ajusta esta ruta según sea necesario

# Cargar datos desde las carpetas de Java, Python y C
java_texts, java_labels = load_data_from_directory(os.path.join(data_directory, 'javaText'))
# python_texts, python_labels = load_data_from_directory(os.path.join(data_directory, 'pythonText'))
# c_texts, c_labels = load_data_from_directory(os.path.join(data_directory, 'cText'))

# Verificar que los datos se han cargado correctamente
print(f'Java texts: {len(java_texts)}')
# print(f'Python texts: {len(python_texts)}')
# print(f'C texts: {len(c_texts)}')

# Unir todos los textos y etiquetas
texts = java_texts #+ python_texts + c_texts
labels = java_labels #+ python_labels + c_labels

# Verificar los tamaños de los textos y etiquetas combinados
print(f'Total texts: {len(texts)}')
print(f'Total labels: {len(labels)}')


# Remover Comentarios
Utilizando expresiones regulares, nos deshacemos de commentarios.

In [ ]:
import re

def remove_comments(text):
    # Remove all block comments (/* */)
    text = re.sub(r'/\*.*?\*/', '', text, flags=re.DOTALL)
    # Remove all line comments (//)
    text = re.sub(r'//.*', '', text)
    # Remove all Python line comments (#)
    text = re.sub(r'#.*', '', text)
    # Remove all Python block comments (''' ''')
    text = re.sub(r"'''(.*?)'''", '', text, flags=re.DOTALL)
    # Remove all Python block comments (""" """)
    text = re.sub(r'"""(.*?)"""', '', text, flags=re.DOTALL)
    
    # Split the text into lines, clean empty lines
    lines = text.split('\n')
    cleaned_lines = [line for line in lines if line.strip() != '']
    
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

# Ejemplo de uso
code = """/* Block comment */
int x = 0; // Line comment

x++; /* Another block
comment */

System.out.println(x); // Print x

# This is a Python comment
y = 2 # Another Python comment

''' Python block comment
spanning multiple lines '''
z = 3; // C style line comment

"""
cleaned_code = remove_comments(code)
print(cleaned_code)


# Calculo de Similitudes
Realizamos el calculo de similitudes con trancisiones de Markov, inicialmente creando la matriz de transcicion a traves del texto, y realizando la similitud de cosenos.

In [ ]:
def create_transition_matrix(text, words):
  # Inicializar la matriz de transición
  matrix = np.zeros((len(words), len(words)))

  # Dividir el texto en palabras
  word_list = text.split()

  # Calcular las frecuencias de las transiciones entre palabras
  for i in range(len(word_list) - 1):
      current_word = word_list[i]
      next_word = word_list[i + 1]

      current_index = words.index(current_word)
      next_index = words.index(next_word)

      matrix[current_index][next_index] += 1

  # Normalizar las filas de la matriz, evitando divisiones por cero
  row_sums = matrix.sum(axis=1)
  nonzero_rows = np.nonzero(row_sums)
  matrix_normalized = np.zeros_like(matrix)
  matrix_normalized[nonzero_rows, :] = matrix[nonzero_rows, :] / row_sums[nonzero_rows, np.newaxis]

  return matrix_normalized

Funcion para obtener archivos.

In [ ]:
def get_file(file):
    return open(file, 'r', encoding='utf-8', errors='ignore').read()

Funcion de separacion de palabras.

In [ ]:
def split_words(text1, text2):
  splitted_words = list(set(text1.split() + text2.split()))
  return splitted_words

Claculo de la similitud de coseno.

In [ ]:
def cos_ang(mat1, mat2):
  mat2T = np.transpose(mat2)
  c = np.dot(mat2T, mat1)
  prod_int = np.trace(c)
  normA = np.sqrt(np.trace((np.dot(np.transpose(mat1), mat1))))
  normB = np.sqrt(np.trace((np.dot(np.transpose(mat2), mat2))))

  return prod_int / (normA * normB)

In [ ]:
def get_words(text):
  file = open(text, "r", encoding = 'utf-8', errors ='ignore')
  words = file.read()
  file.close()

  punctuation = ["_", "-", ".", ":", "," , ";", "(", ")", "?", "¿", "¡", "!"]
  for i in punctuation:
      words = words.replace(i, " ")

  words = words.lower()
  words = words.replace('\n', ' ')

  return words

# Generar Tabla de Similitudes
Generamos nuestra tabla de similitudes/Realiza la evaluacion de la similitud de textos utilizando las funciones anteriores.

In [ ]:
def similitud(text1, text2):
  data1 = remove_comments(text1)
  data2 = remove_comments(text2)

  split_data = split_words(data1, data2)

  mat1 = create_transition_matrix(data1, split_data)
  mat2 = create_transition_matrix(data2, split_data)
  cos = cos_ang(mat1, mat2)

  return cos # Debe retornar un valor %cos

Obtenemos pruebas aleatoreamente del Dataset.

In [ ]:
def get_two_random_texts(texts):

  if len(texts) <= 1:
    raise ValueError("The provided list of texts must contain at least two elements.")

  first_text_index = random.randint(0, len(texts) - 1)
  second_text_index = random.randint(0, len(texts) - 1)

  # Ensure distinct texts are returned
  while first_text_index == second_text_index:
    second_text_index = random.randint(0, len(texts) - 1)

  return texts[first_text_index],texts[second_text_index]


In [ ]:
similitud(get_two_random_texts(texts)[0], get_two_random_texts(texts)[1])

Agregamos una columna de Similitud dependiendo del Resultado, y una identificada como Plagio si la similitud es mayor al 80%.

In [ ]:
df['Similitud'] = df.apply(lambda row: similitud(row['Code1'], row['Code2']), axis=1)
threshold = 0.8
df['Plagio'] = (df['Similitud'] > threshold).astype(int)
df

Guardamos nuestra tabla en CSV para ser leida por nuestros modelos.

In [ ]:
df.to_csv('data.csv', index=False)

Visualizamos la distribución de los datos.

In [1]:
# Visualizar la distribución de similitudes
plt.figure(figsize=(10, 6))
plt.hist(df['Similitud'], bins=20, color='blue', edgecolor='black')
plt.title('Distribución de Similitudes')
plt.xlabel('Similitud')
plt.ylabel('Frecuencia')
plt.show()

NameError: name 'plt' is not defined